In [11]:
import torch
# print(torch.cuda.is_available())
# print(torch.cuda.device_count())
# True
# 1


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [13]:
# Inception

class inception(nn.Module):
    def __init__(self, in_channels):
        super(inception, self).__init__()

        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

        # 16+24+24+24 = 88


    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = torch.cat([branch1x1, branch5x5, branch3x3, branch_pool], dim=1)
        return outputs

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  # MNIST (1x28x28) -> (10x24x24)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.inception1 = inception(in_channels=10)
        self.inception2 = inception(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x))) # (1x28x28) -> (10x24x24) -> (10x12x12)
        x = self.inception1(x) # (10x12x12) -> (88x12x12)
        x = F.relu(self.mp(self.conv2(x))) # (88x12x12) ->(20x8x8) -> (20x4x4)
        x = self.inception2(x) # (20x4x4) -> (88x4x4) -> 1408
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

In [15]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Epoch 1/10, Loss: 0.1864
Epoch 2/10, Loss: 0.0634
Epoch 3/10, Loss: 0.0480
Epoch 4/10, Loss: 0.0429
Epoch 5/10, Loss: 0.0374
Epoch 6/10, Loss: 0.0353
Epoch 7/10, Loss: 0.0301
Epoch 8/10, Loss: 0.0292
Epoch 9/10, Loss: 0.0253
Epoch 10/10, Loss: 0.0236
Accuracy on test set: 99.01%
